# Import Library

In [22]:
# Import all the libraries I need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ignore Deprecation Warning
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning) 

from sklearn.ensemble import RandomForestRegressor

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential # intitialize the ANN
from tensorflow.keras.layers import Dense      # create layers
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import Binarizer
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# load the data
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df = df_train.append(df_test , ignore_index = True)

Using TensorFlow backend.


# Pre-processing
I use class to make easy to try different model and pre-rocessing, so it looks a little be wired.   

In [23]:
class EDA1(): 
    def __init__(self) -> None:
        self.X_train, self.y_train,self.X_test=[],[],[]
    #Data pre-processing  
    def data_handle(self,df,df_train):
        
        # Processing Title：according to the passenger's name, dividing the passengers into five categories: Miss, Mrs, Master, Mr, Others 
        # and store these five categories as features in the data in the form of one-hot encoding.
        df['Title'] = df.Name.map( lambda x: x.split(',')[1].split( '.' )[0].strip())
        df['Title'] = df['Title'].replace('Mlle', 'Miss')
        df['Title'] = df['Title'].replace(['Mme','Lady','Ms'], 'Mrs')
        df.Title.loc[ (df.Title !=  'Master') & (df.Title !=  'Mr') & (df.Title !=  'Miss') 
                    & (df.Title !=  'Mrs')] = 'Others'
        df = pd.concat([df, pd.get_dummies(df['Title'])], axis=1).drop(labels=['Name'], axis=1)

        # map the two genders to 0 and 1
        df.Sex = df.Sex.map({'male':0, 'female':1})

        # In revious analisis, the family Size is important feature to train. So here we create a new feature "Family" to store it.
        df['Family'] = df['SibSp'] + df['Parch'] + 1
        df.Family = df.Family.map(lambda x: 0 if x > 4 else x)

        # It is because that the value of ticket mix with text and number, 
        # so we get first character in ticket then classify iten 1-4 and S, P, C
        df.Ticket = df.Ticket.map(lambda x: x[0])
        guess_Fare = df.Fare.loc[ (df.Ticket == '3') & (df.Pclass == 3) & (df.Embarked == 'S')].median()
        df.Fare.fillna(guess_Fare , inplace=True)
        df['Fare-bin'] = pd.qcut(df.Fare,5,labels=[1,2,3,4,5]).astype(int)
        df = df.drop(labels=['Cabin'], axis=1)  # delete the feature we don't really need
        
        # fill the NAN
        df.Embarked.fillna('S' , inplace=True )
        df = df.drop(labels='Embarked', axis=1)
        
        # notice that instead of using Title, we should use its corresponding dummy variables 
        df_sub = df[['Age','Master','Miss','Mr','Mrs','Others','Fare-bin','SibSp']]

        self.X_train  = df_sub.dropna().drop('Age', axis=1) #X_train doesn't have Age column [1046,8]
        self.y_train  = df['Age'].dropna() #[1046 17]
        self.X_test = df_sub.loc[np.isnan(df.Age)].drop('Age', axis=1)

        # predict the age to fill the NAN
        regressor = RandomForestRegressor(n_estimators = 500) 
        regressor.fit(self.X_train, self.y_train)
        y_pred = np.round(regressor.predict(self.X_test),1)
        df.Age.loc[df.Age.isnull()] = y_pred

        df.Age.isnull().sum(axis=0) # no more NAN now

        bins = [ 0, 4, 12, 18, 30, 50, 65, 100] # This is somewhat arbitrary...
        age_index = (1,2,3,4,5,6,7)
        #('baby','child','teenager','young','mid-age','over-50','senior')
        df['Age-bin'] = pd.cut(df.Age, bins, labels=age_index).astype(int)

        df['Ticket'] = df['Ticket'].replace(['A','W','F','L','5','6','7','8','9'], '4')
        # dummy encoding
        df = pd.get_dummies(df,columns=['Ticket'])
        df = df.drop(labels=['SibSp','Parch','Age','Fare','Title'], axis=1)

        self.y_train = df[0:891]['Survived'].values
        self.X_train = df[0:891].drop(['Survived','PassengerId'], axis=1).values
        self.X_test  = df[891:].drop(['Survived','PassengerId'], axis=1).values 
        
        
    def Predict(self,model,batch_size):
        x = model.predict(self.X_test,batch_size=batch_size)
        return x

In [24]:
class Models():
    def __init__(self) -> None:
        pass
    def Opt_gen(self,lr = 0.01,momentum = 0.8,epochs = 10,opt_type = "SGD"):
        if opt_type is "SGD":
            print("SGD!",end="")
            decay = lr / epochs
            opt = keras.optimizers.SGD(learning_rate = lr, decay = decay, momentum = momentum, nesterov=False) 
        else:
            print(opt_type,end="")
            return opt_type
        return opt
    def create_model(self,lyrs = [9,9,5], opt = "Nadam", activation = 'selu',lr=0.01,epochs=10,kernel_init = "random_normal",
        base_init=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),X_train = df_train):
            
            base_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
            model = Sequential()
            # First layer
            model.add(Dense(lyrs[0],bias_initializer=base_init, kernel_initializer = kernel_init, activation = activation, input_dim = X_train.shape[1]))
            
            # layers
            for i in range(1,len(lyrs)):
                model.add(Dense(lyrs[i],bias_initializer=base_init, activation=activation,kernel_initializer = kernel_init))

            # output layer
            model.add(Dense(units = 1,bias_initializer=base_init, kernel_initializer = kernel_init, activation = 'sigmoid'))

            optimizers = self.Opt_gen(lr=lr,opt_type=opt,epochs=epochs)
            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['accuracy'])
            model.summary()
            return model
   
Model=Models()

In [25]:
# draw the picture after training
def draw(history):
    val_acc = np.mean(history.history['val_accuracy'])
    print("\n%s: %.2f%%" % ('val_acc', val_acc*100))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation','loss','val_loss'], loc='upper left')
    plt.show()

# Start training

In [ ]:

My_data = EDA1()
My_data.data_handle(df,df_train)

epochs = 100
batch_size = 10
stop = EarlyStopping(monitor='val_loss', patience = epochs, verbose = 1)

### age
model = Model.create_model(lr=0.001,activation="selu",base_init="zero",opt="SGD",kernel_init="he_normal",X_train=My_data.X_train)
history = model.fit(My_data.X_train,My_data.y_train, epochs=epochs, batch_size=batch_size, validation_split=0.33, shuffle = True,callbacks = [stop])
draw(history)
## Perdict
prediction_reg=My_data.Predict(model,batch_size)

# Save the prediction

In [27]:
def save(prediction,name):
    binarizer = Binarizer(threshold = 0.5)
    prediction = binarizer.fit_transform(prediction)
    prediction = prediction.astype(np.int32)                       
    # create submission file
    PassengerId = df_test["PassengerId"]
    evaluation = PassengerId.to_frame()
    evaluation["Survived"] = prediction
    evaluation.to_csv(f"{name}.csv", index = False)

In [28]:
## save
save(prediction_reg,"submission")

# Other Part

## GridSearch work

In [ ]:
# GridSearchCV 
modelCV = KerasClassifier(build_fn=model, verbose=0)

# define the grid search parameters
batch_size_buf = [8]
epochs_buf = [20]
optimizer_buf = ['SGD', 'Adagrad', 'Adadelta', 'Adam', 'Nadam']
# layers = [(9,9,5),(13,10,5)]
activation = ['softmax', 'softplus', 'relu', 'tanh','selu']
param_grid = dict(batch_size=batch_size_buf, epochs=epochs_buf,opt=optimizer_buf,activation=activation) 

# search the grid
grid = GridSearchCV(estimator=modelCV, 
                    param_grid=param_grid,
                    cv=3,
                    verbose=2)  # include n_jobs=-1 if you are using CPU

grid_result = grid.fit(My_data.X_train, My_data.y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))